## Download the Summarization Model (if not present in `models/`)

In [1]:
import os
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

model_dir = 'models' 
model_filename = 'Meta-Llama-3.1-70B-Instruct.IQ3_XS.gguf'
token = os.environ['HF_TOKEN']

if not os.path.exists('models'):
    os.mkdir('models')

if model_filename not in os.listdir(model_dir):
    print(f"Downloading and storing the summarizatin model in: {os.path.join(model_dir, model_filename)}")
    print(f"Using the following Hugging Face token: {token}")
    
    file_path = hf_hub_download(
        repo_id='lmstudio-community/Meta-Llama-3.1-70B-Instruct-GGUF',
        filename=model_filename,
        local_dir=model_dir,
        use_auth_token=token,
    )

In [2]:
model_path = os.path.join(model_dir, model_filename)

llm = Llama(
    model_path=model_path,
    n_gpu_layers=33,  # tuen based on your GPU VRAM availability
    n_ctx=8129, # tune based on the dialogue text size
    n_threads=40,
    threads_batch=2,
    verbose=True,
    use_tensor_cores=True,
)

llama_model_loader: loaded meta data with 33 key-value pairs and 723 tensors from models/Meta-Llama-3.1-70B-Instruct.IQ3_XS.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models Meta Llama Meta Llama 3.1 70B ...
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = models-meta-llama-Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 70B
llama_model_loader: - kv   6:                            general.license str              = lla

In [3]:
from IPython.display import display, clear_output

def load_text_file(path: os.PathLike) -> str:
    with open(path, 'r') as file:
        return file.read()

def summarize_text(text: str) -> str:
    output = ''
    response_generator = llm.create_chat_completion(
        messages = [
            {
                "role": "user",
                "content": f"""
                    Please, summarize the following text extracted from a document: {text}
                    It is important to highlight the main information!
                    The produced text must be in the same language as the original!
                    You may add details and complement some information when appropriate.
                    """
            }
        ],
        stream=True,
    )
    
    for r in response_generator:
        clear_output(wait=True)
        if 'content' in r['choices'][0]['delta']:
            output += r['choices'][0]['delta']['content']
        print(output)
    return output

In [7]:
files_to_summarize = ['../outputs/digital-thermometer-pdfplumber.md']
summaries = {file: '' for file in files_to_summarize}

for file in files_to_summarize:
    text = load_text_file(path=file)
    summaries[file] = summarize_text(text)

The DS18B20 is a digital thermometer that provides 9-bit to 12-bit Celsius temperature measurements and has an alarm function with nonvolatile user-programmable upper and lower trigger points. It communicates over a 1-Wire bus that requires only one data line for communication with a central microprocessor. The device can derive power directly from the data line, eliminating the need for an external power supply.

**Key Features:**

* 9-bit to 12-bit Celsius temperature measurements
* Nonvolatile user-programmable upper and lower trigger points
* 1-Wire bus communication
* Parasite power mode
* Unique 64-bit serial code for each device

**Applications:**

* Thermostatic controls
* Industrial systems
* Consumer products
* Thermometers
* Thermally sensitive systems

**Benefits:**

* Unique 1-Wire interface requires only one port pin for communication
* Reduces component count with integrated temperature sensor and EEPROM
* Measures temperatures from -55°C to +125°C
* ±0.5°C accuracy from